In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [3]:
df=pd.read_csv("C:\\Users\\vedan\\OneDrive\\Desktop\\Churn_Modelling.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df=df.dropna()

In [6]:
from sklearn import preprocessing
my_label=preprocessing.LabelEncoder()
df['Gender']=my_label.fit_transform(df['Gender'])

In [7]:
x=df[['CreditScore','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]
y=df['Exited']

In [8]:
print(x.shape)
print(y.shape)

(10000, 9)
(10000,)


In [9]:
model=Sequential([
    tf.keras.Input(shape=(9,)),
    Dense(512,activation='relu'),
    Dense(256,activation='relu'),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(1,activation='sigmoid'),
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               5120      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 177,665
Trainable params: 177,665
Non-trainable params: 0
_________________________________________________________________


In [11]:
[layer1, layer2, layer3,layer4,layer5] = model.layers

In [12]:
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
W4,b4 = layer4.get_weights()
W5,b5 = layer5.get_weights()

In [1]:
model.compile(
    loss='BinaryCrossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x,y,
    epochs=20
)

NameError: name 'model' is not defined

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state = 1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(7000, 9) (3000, 9) (7000,) (3000,)


In [15]:
predictions=model.predict(x_test)

94/94 [==============================] - 0s 2ms/step


In [16]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, yhat))

0.791


In [18]:
from sklearn.svm import SVC 
classifier = SVC(kernel='linear', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(kernel='linear', random_state=0)

In [19]:
y_pred= classifier.predict(x_test) 

In [20]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  

In [21]:
print(cm)

[[2300   73]
 [ 605   22]]


In [31]:
df.drop("Surname",axis = 1,inplace = True)
df.drop("Geography",axis = 1,inplace = True)

In [32]:
data_meaned = df - np.mean(df, axis = 0)

cov_mat = np.cov(data_meaned , rowvar = False)

eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)

sorted_index = np.argsort(eigen_values)[::-1]

sorted_eigenvalue = eigen_values[sorted_index]

sorted_eigenvectors = eigen_vectors[:,sorted_index]

n_components = 2

eigenvector_subset = sorted_eigenvectors[:,0:n_components]

data_reduced = np.dot(eigenvector_subset.transpose(),data_meaned.transpose()).transpose()

In [34]:
data_reduced

array([[ 52996.28257809,  78278.58711974],
       [ 43470.02099602,  -6599.91942399],
       [ 74587.09002971, -81150.74810165],
       ...,
       [104929.48277548,  85115.29730263],
       [  8748.34560365,   2335.50887137],
       [ 66814.19636303, -45889.78709654]])

In [35]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

In [36]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2372    1]
 [ 626    1]]
